Merge your TSO + DSO Wind capacity files
Add Zone mapping (East / West)
Group & sum by Zone, Type, Connection_Type
Save both the merged raw file & the grouped zone table

In [21]:
import pandas as pd

In [48]:
# === 1️⃣ File paths ===
tso_file = '../data/processed/TSO_Wind_Capacity_Clean.csv'
dso_file = '../data/processed/DSO_Wind_Capacity_Clean.csv'
merged_output = '../data/processed/Wind_Capacity_Master.csv'
regional_output = '../data/processed/Regional_Wind_Capacity.csv'

In [23]:
# === 2️⃣ Load TSO & DSO ===
df_tso = pd.read_csv(tso_file)
df_dso = pd.read_csv(dso_file)

In [32]:
# === 3️⃣ If DSO doesn't have Category, add it ===
if 'Category' not in df_dso.columns:
    df_dso['Category'] = 'Onshore'

In [33]:
# === 4️⃣ Merge both ===
df_merged = pd.concat([df_tso, df_dso], ignore_index=True)

In [39]:
# === 5️⃣ Define Republic-only zones ===
east_counties = [
    "Dublin", "Meath", "Wicklow", "Louth", "Kildare", "Offaly",
    "Laois", "Carlow", "Wexford", "Kilkenny", "Longford", "Westmeath"
]

west_counties = [
    "Galway", "Mayo", "Clare", "Sligo", "Leitrim", "Roscommon"
]

north_counties = [
    "Donegal", "Cavan", "Monaghan"
    # Derry, Tyrone removed — those will be filtered as NI below
]

south_counties = [
    "Cork", "Kerry", "Waterford", "Tipperary", "Limerick"
]

In [40]:
# === 6️⃣ NI counties to filter out ===
ni_counties = ["Derry", "Tyrone", "Antrim", "Down", "Armagh", "Fermanagh"]

In [41]:
# === 7️⃣ Map County → Zone ===
def county_to_zone(county):
    if county in ni_counties:
        return 'NI'
    elif county in east_counties:
        return 'East'
    elif county in west_counties:
        return 'West'
    elif county in north_counties:
        return 'North'
    elif county in south_counties:
        return 'South'
    else:
        return 'Other'


In [42]:
df_merged['Zone'] = df_merged['County'].apply(county_to_zone)

In [43]:
# === 8️⃣ Inspect any that ended up "Other" for QC
print("\n✅ Counties mapped as 'Other':")
print(df_merged[df_merged['Zone'] == 'Other']['County'].unique())


✅ Counties mapped as 'Other':
[]


In [44]:
# === 9️⃣ Save full Wind merged block (keeps NI + Other for traceability)
df_merged.to_csv(merged_output, index=False)
print(f"✅ Wind capacity master saved to: {merged_output}")

✅ Wind capacity master saved to: ../data/processed/Wind_Capacity_Master.csv


In [45]:
# === 🔟 Filter out NI rows for regional grouping
df_regional = df_merged[df_merged['Zone'] != 'NI']

In [46]:
# === 1️⃣1️⃣ Group by Zone, Type, Connection_Type & sum Installed MW
df_regional = (
    df_regional.groupby(['Zone', 'Type', 'Connection_Type'])
    .agg({'Installed_MW': 'sum'})
    .reset_index()
)

In [47]:
# === 1️⃣2️⃣ Save Regional Wind Capacity table
df_regional.to_csv(regional_output, index=False)
print(f"✅ Regional wind capacity saved to: {regional_output}")


✅ Regional wind capacity saved to: ../data/processed/Regional_Wind_Capacity.csv
